# Pileup and Jet ID

## Loading packages and input files

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import uproot
import numpy as np
import correctionlib

plt.style.use(hep.style.CMS)
plt.rcParams.update({
    'font.size': 16,         # Base font size
    'axes.titlesize': 18,    # Title font size
    'axes.labelsize': 16,    # Axis label font size
    'xtick.labelsize': 14,   # X-tick label size
    'ytick.labelsize': 14,   # Y-tick label size
    'legend.title_fontsize': 16, # Legend title size
    'legend.fontsize': 16        # Legend font size
})

NanoAODSchema.warn_missing_crossrefs = False

In [ ]:
fname = "root://eosuser.cern.ch//eos/user/c/cmsdas/2025/jets/RunIISummer20UL18NanoAODv9/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/2520000/47A0D984-2D6F-0D42-B5CB-906786434349.root"
events = NanoEventsFactory.from_root(
    file=fname,
    schemaclass=NanoAODSchema,
    metadata={"dataset": "TTJets_JMENano"},
).events()

## Exercise 2.1: Pileup event variables

Let's make histograms of different pileup event variables and look at their distributions:

In [ ]:
hists = (
    hist.Hist.new
    .Reg(100, 0, 101, name="npu", label='NPU')
    .Reg(100, 0, 101, name="mu", label='mu')
    .Reg(100, 0, 101, name="rho", label='rho')
    .Reg(100, 0, 101, name="npv", label='NPV')
    .Weight()
    .fill(
        npu=events.Pileup.nPU,
        mu=events.Pileup.nTrueInt,
        rho=events.fixedGridRhoFastjetAll,
        npv= events.PV.npvs
    )
)

In [ ]:
hists.project('mu', 'npv').plot2d()

In [ ]:
hists.project('npv', 'rho').plot2d()

In [ ]:
hists.project('mu', 'rho').plot2d()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

hists.project('mu').plot1d(ax=ax, density=True, label=r"\mu")
hists.project('rho').plot1d(ax=ax, density=True, label=r'\rho')
hists.project('npv').plot1d(ax=ax, density=True, label='NPV')
hists.project('npu').plot1d(ax=ax, density=True, label='NPU')

ax.legend()

## Exercise 2.2: CHS vs PUPPI

Let's play with some basic quantities between CHS and PUPPI.

In [ ]:
jets = events.Jet[ (events.Jet.pt > 30) ]
jets_puppi = events.JetPuppi[ (events.JetPuppi.pt > 30) ]

hists = (
    hist.Hist.new
    .StrCategory(['chs', 'puppi'], name='pu', growth=True)
    .Reg(50, 0, 500, name="pt", label='pt')
    .Reg(40, -5, 5, name="eta", label='eta')
    .Weight()
    .fill(
        pu='chs',
        pt=ak.flatten(jets.pt) ,
        eta=ak.flatten(jets.eta)
    )
    .fill(
        pu='puppi',
        pt=ak.flatten(jets_puppi.pt),
        eta=ak.flatten(jets_puppi.eta)
    )
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
hists.project('pu', 'pt').plot1d(ax=ax)
ax.legend()
ax.set_yscale("log")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
hists.project('pu', 'eta').plot1d(ax=ax)
ax.legend()
# ax.set_yscale("log")

## Exercise 2.3: Pileup Reweighting 

Let's use again the `json-pog` and `correctionlib` to access the pileup weights:

In [ ]:
correction_file = "/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/LUM/2018_UL/puWeights.json.gz"
pu_weight_corr = list(correctionlib.CorrectionSet.from_file(correction_file).values())[0]

In [ ]:
nTrueInt_array = np.array(events.Pileup.nTrueInt)
pu_weight = pu_weight_corr.evaluate(nTrueInt_array, 'nominal')

Now let's look at the PU weights as a function of $\mu$

In [ ]:
hists = (
    hist.Hist.new
    .Reg(100, 0, 100, name="pu")
    .Reg(200, 0, 2, name="puweight")
    .Weight()
    .fill(
        puweight=pu_weight,
        pu=events.Pileup.nTrueInt.to_numpy()
    )
)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 8))

axs.axhline(y=1, color='gray', linestyle='--')

axs.errorbar(
    x= hists.axes[0].centers,
    y= hists.profile('puweight').values(),
    yerr= abs(hists.profile('puweight').variances()),
    xerr=np.ones(len(hists.axes[0].centers))*0.5,
    marker='v', linestyle="",
)

axs.set_xlabel('Number of interactions')
axs.set_ylabel('PU weight')

Let's use these weights in a distribution, like the number of primary vertices, to see their effect:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

hists = (
    hist.Hist.new
    .Reg(99, 0, 100, name="npv")
    .Weight()
    .fill(
        npv=events.PV.npvs.to_numpy(),
        weight=pu_weight,
    )
).plot1d(density=True, ax=ax, label='w PUweight')

hists = (
    hist.Hist.new
    .Reg(99, 0, 100, name="npv")
    .Weight()
    .fill(
        npv=events.PV.npvs.to_numpy(),
    )
).plot1d(density=True, ax=ax, label="wo PUweight")

ax.legend()

## Exercise 2.4: Jet ID

Let's create a histogram with the jetId information:

In [ ]:
hists3 = (
    hist.Hist.new
    .Reg(50, 0, 500, name="pt")
    .Reg(40, -5, 5, name="eta")
    .Reg(10, 0, 10, name="jetId")
    .Weight()
    .fill(
        pt=ak.flatten(events.Jet.pt),
        eta=ak.flatten(events.Jet.eta),
        jetId=ak.flatten(events.Jet.jetId),
    )
)

In [ ]:
hists3.project("jetId").plot1d()

Let's apply a simple selection to the `events.Jet` collection to remove those jets:

In [ ]:
jets = events.Jet[events.Jet.jetId>=2]

hists4 = (
    hist.Hist.new
    .Reg(50, 0, 500, name="pt")
    .Reg(40, -5, 5, name="eta")
    .Reg(10, 0, 10, name="jetId")
    .Weight()
    .fill(
        pt=ak.flatten(jets.pt),
        eta=ak.flatten(jets.eta),
        jetId=ak.flatten(jets.jetId),
    )
)

In [ ]:
hists4.project("jetId").plot1d()

## Your turn

How different do the other jet distributions look like with jetId implemented?